In [1]:
# libraries
import numpy as np
import pandas as pd

In [2]:
# read in data frames
election_38_df = pd.read_csv("data/merged/38_general_election.csv")
election_39_df = pd.read_csv("data/merged/39_general_election.csv")
election_40_df = pd.read_csv("data/merged/40_general_election.csv")
election_41_df = pd.read_csv("data/merged/41_general_election.csv")
election_42_df = pd.read_csv("data/merged/42_general_election.csv")
election_43_df = pd.read_csv("data/merged/43_general_election.csv")
election_44_df = pd.read_csv("data/merged/44_general_election.csv")

In [9]:
# metadata for the elections

election_caps_dict = {
    "38": 5000,
    "39": 5000,
    "40": 1000,
    "41": 1000,
    "42": 1500,
    "43": 1500,
    "44": 1500
}

election_years_dict = {
    "38": 2004,
    "39": 2006,
    "40": 2008,
    "41": 2011,
    "42": 2015,
    "43": 2019,
    "44": 2021
}

metadata_elections_df = pd.DataFrame({
    "election": election_caps_dict.keys(),
    "contribution_cap": election_caps_dict.values(),
    "election_year": election_years_dict.values()
})

metadata_elections_df

,election,contribution_cap,election_year
0,38,5000,2004
1,39,5000,2006
2,40,1000,2008
3,41,1000,2011
4,42,1500,2015
5,43,1500,2019
6,44,1500,2021


In [10]:
# grab CPI for inflation adjustment
cpi_df = pd.read_csv('data/cpi.csv')
cpi_df = cpi_df[cpi_df['Products and product groups'] == 'All-items'][['REF_DATE', 'VALUE']]

# rebase the CPI to 2004 = 100, since the data is using 2002 = 100
base_2004_cpi = cpi_df.loc[cpi_df['REF_DATE'] == 2004]['VALUE'][0]
cpi_df['VALUE'] = cpi_df['VALUE']/base_2004_cpi*100
cpi_df.head()

,REF_DATE,VALUE
0,2004,100.000000
1,2005,102.196753
2,2006,104.202483
3,2007,106.494747
4,2008,108.978032


In [11]:
metadata_elections_df = metadata_elections_df.copy().merge(cpi_df, left_on='election_year', right_on='REF_DATE')
metadata_elections_df['election_name'] = [
    "38th General Election",
    "39th General Election",
    "40th General Election",
    "41st General Election",
    "42nd General Election",
    "43rd General Election",
    "44th General Election"
]
metadata_elections_df

,election,contribution_cap,election_year,REF_DATE,VALUE,election_name
0,38,5000,2004,2004,100.000000,38th General Election
1,39,5000,2006,2006,104.202483,39th General Election
2,40,1000,2008,2008,108.978032,40th General Election
3,41,1000,2011,2011,114.517670,41st General Election
4,42,1500,2015,2015,120.916905,42nd General Election
5,43,1500,2019,2019,129.894938,43rd General Election
6,44,1500,2021,2021,135.243553,44th General Election


In [12]:
metadata_elections_df.to_csv('data/power_bi/elections_metadata.csv')

In [33]:
# setting up preprocessing for all the frames

columns_rename_map = {
    "Recipient ID": "recipient_id",
    "Political Party of Recipient": "political_party",
    "ED_CODE": "electoral_district_code", #use the code instead, need to remember to set this as categorical
    "Monetary amount": "monetary_amount",
    "Percentage of Votes Obtained /Pourcentage des votes obtenus": "percentage_vote"
}

def preprocess_raw(df, election_number):
   # drop unneeded columns
   df = df.drop(columns=['Unnamed: 0', 'Political Entity', 'Fiscal/Election date', 
                        'Recipient last name', 'Recipient first name', 'Contribution Received date',
                        'cleaned_district_name_x', 'cleaned_district_name'], 
                errors='ignore')

   # remove any 0 monetary amounts - these are probably residuals from the non-monetary amounts, not relevant to our modelling here 
   df = df[df['Monetary amount'] != 0]

   # rename columns for consistency and ease 
   df = df.rename(columns=columns_rename_map, errors = 'ignore')

   # add in election number
   df['election_number'] = election_number

   
   return df

def preprocess_summary(df, election_number):
   
   # drop unneeded columns
   df = df.drop(columns=['Unnamed: 0', 'Political Entity', 'Electoral event', 'Fiscal/Election date', 
                        'Recipient last name', 'Recipient first name','Contributor City', 'Contributor Province', 
                        'Contributor Postal code', 'Contribution Received date', 'cleaned_district_name_x',], 
                errors='ignore')
    
   # remove any 0 monetary amounts - these are probably residuals from the non-monetary amounts, not relevant to our modelling here 
   df = df[df['Monetary amount'] != 0]

   # rename columns for consistency and ease 
   df = df.rename(columns=columns_rename_map, errors = 'ignore')

   # adjust for inflation, normalize to the contribution cap
   cpi = metadata_elections_df[metadata_elections_df['election'] == election_number]['VALUE'].iloc[0]
   year = metadata_elections_df[metadata_elections_df['election'] == election_number]['election_year'].iloc[0]
   cap = metadata_elections_df[metadata_elections_df['election'] == election_number]['contribution_cap'].iloc[0]

   df['monetary_amount'] = df['monetary_amount'] *cpi/100
   



   # groupby and agg
   df = df.groupby('recipient_id').agg(
      # aggregate functions 
      monetary_sum = pd.NamedAgg(column= 'monetary_amount', aggfunc="sum"), 
      num_donations = pd.NamedAgg(column= 'monetary_amount', aggfunc='count'),
      monetary_mean = pd.NamedAgg(column= "monetary_amount", aggfunc = 'mean'),
      #monetary_std = pd.NamedAgg(column= 'monetary_amount', aggfunc= 'std'),
      monetary_min = pd.NamedAgg(column= 'monetary_amount', aggfunc= 'min'),
      monetary_max = pd.NamedAgg(column= 'monetary_amount', aggfunc= 'max'),
      #num_max_donations = pd.NamedAgg(column='monetary_amount', aggfunc=lambda x: (x >= 0.5).sum()),

      # agg for the other columns
      # political_party = pd.NamedAgg(column= 'political_party', aggfunc= 'first'),
      electoral_district = pd.NamedAgg(column = 'electoral_district', aggfunc= 'first'),
      # percentage_vote = pd.NamedAgg(column = 'percentage_vote', aggfunc = 'first'),
      #election_number = pd.NamedAgg(column = 'election_number', aggfunc = 'first'),
      #won_election = pd.NamedAgg(column = 'won_election', aggfunc= 'first')

   ).reset_index()

   # add in election number, year, cap
   df['election_number'] = election_number
   df['election_year'] = year
   df['contribution_cap'] = cap

   # calculate the total amount of donations within a district per election
   district_total = df.groupby(['electoral_district', 'election_number'])['monetary_sum'].transform('sum')
   df['district_share'] = df['monetary_sum']/district_total

   # calculated the percent of max donations a candidate hits
   #df['max_donation_pct'] = df['num_max_donations']/df['num_donations']
 
   # drop the recipent_id
   df = df.drop(columns= ['recipient_id'])
    
   return df

In [4]:
election_38_df

,Unnamed: 0,Political Entity,Recipient ID,Recipient,Recipient last name,Recipient first name,Political Party of Recipient,Electoral District,Electoral event,Fiscal/Election date,Contributor City,Contributor Province,Contributor Postal code,Contribution Received date,Monetary amount,cleaned_district_name_x,ED_CODE,Percentage of Votes Obtained /Pourcentage des votes obtenus,won_election
0,0,﻿Candidates,4716,"Béchard, Bruno-Marie",Béchard,Bruno-Marie,Liberal Party of Canada,Sherbrooke,38th general election,2004-06-28,Magog,QC,J1X 2C3,NaN,400.0,sherbrooke,24070.0,31.01,False
1,1,Candidates,4716,"Béchard, Bruno-Marie",Béchard,Bruno-Marie,Liberal Party of Canada,Sherbrooke,38th general election,2004-06-28,Sherbrooke,QC,J1L 2B6,NaN,500.0,sherbrooke,24070.0,31.01,False
2,2,Candidates,4716,"Béchard, Bruno-Marie",Béchard,Bruno-Marie,Liberal Party of Canada,Sherbrooke,38th general election,2004-06-28,Ascot,QC,J1K 3B4,NaN,500.0,sherbrooke,24070.0,31.01,False
3,3,Candidates,4716,"Béchard, Bruno-Marie",Béchard,Bruno-Marie,Liberal Party of Canada,Sherbrooke,38th general election,2004-06-28,Shebrooke,QC,J1H 4J9,NaN,2500.0,sherbrooke,24070.0,31.01,False
4,4,Candidates,4716,"Béchard, Bruno-Marie",Béchard,Bruno-Marie,Liberal Party of Canada,Sherbrooke,38th general election,2004-06-28,Katevale,QC,J0B 1W0,NaN,500.0,sherbrooke,24070.0,31.01,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18501,18501,Candidates,5250,"Carter, Roger",Carter,Roger,Marxist-Leninist Party of Canada,Beaches--East York,38th general election,2004-06-28,NaN,NaN,NaN,NaN,0.0,beaches east york,35005.0,0.10,False
18502,18502,Candidates,5615,"Giguère, Philippe",Giguère,Philippe,New Democratic Party,Beauce,38th general election,2004-06-28,NaN,NaN,NaN,NaN,75.0,beauce,24005.0,3.05,False
18503,18503,Candidates,5141,"Morrison, Dean",Morrison,Dean,New Democratic Party,Langley,38th general election,2004-06-28,NaN,NaN,NaN,NaN,0.0,langley,59013.0,16.75,False
18504,18504,Candidates,4549,"Paradis, Denis",Paradis,Denis,Liberal Party of Canada,Brome--Missisquoi,38th general election,2004-06-28,NaN,NaN,NaN,NaN,0.0,brome missisquoi,24010.0,42.08,True


In [31]:
election_38_processed_df = preprocess_raw(election_38_df, "38")
election_39_processed_df = preprocess_raw(election_39_df, "39")
election_40_processed_df = preprocess_raw(election_40_df, "40")
election_41_processed_df = preprocess_raw(election_41_df, "41")
election_42_processed_df = preprocess_raw(election_42_df, "42")
election_43_processed_df = preprocess_raw(election_43_df, "43")
election_44_processed_df = preprocess_raw(election_44_df, "44")


raw_df = pd.concat([election_38_processed_df, election_39_processed_df, election_40_processed_df, 
           election_41_processed_df, election_42_processed_df, election_43_processed_df, election_44_processed_df], axis= 0)

raw_df.to_csv('data/power_bi/elections_raw.csv')
raw_df

,recipient_id,Recipient,political_party,Electoral District,Electoral event,Contributor City,Contributor Province,Contributor Postal code,monetary_amount,electoral_district,percentage_vote,won_election,election_number
0,4716,"Béchard, Bruno-Marie",Liberal Party of Canada,Sherbrooke,38th general election,Magog,QC,J1X 2C3,400.0,24070.0,31.01,False,38
1,4716,"Béchard, Bruno-Marie",Liberal Party of Canada,Sherbrooke,38th general election,Sherbrooke,QC,J1L 2B6,500.0,24070.0,31.01,False,38
2,4716,"Béchard, Bruno-Marie",Liberal Party of Canada,Sherbrooke,38th general election,Ascot,QC,J1K 3B4,500.0,24070.0,31.01,False,38
3,4716,"Béchard, Bruno-Marie",Liberal Party of Canada,Sherbrooke,38th general election,Shebrooke,QC,J1H 4J9,2500.0,24070.0,31.01,False,38
4,4716,"Béchard, Bruno-Marie",Liberal Party of Canada,Sherbrooke,38th general election,Katevale,QC,J0B 1W0,500.0,24070.0,31.01,False,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6823,48968,"Blais, Gloriane",Independent,Mégantic--L'Érable,44th general election,NaN,NaN,NaN,35.0,24047.0,0.90,False,44
6825,47567,"Cook, Anthony",People's Party of Canada,Vancouver South,44th general election,NaN,NaN,NaN,20.0,59040.0,2.70,False,44
6826,47402,"Wright, Michael",Green Party of Canada,Regina--Lewvan,44th general election,NaN,NaN,NaN,30.0,47007.0,1.20,False,44
6827,47647,"Hickey, Jason",Liberal Party of Canada,New Brunswick Southwest,44th general election,NaN,NaN,NaN,20.0,13008.0,23.90,False,44


In [25]:
election_38_processed_df = preprocess_summary(election_38_df, "38")
election_39_processed_df = preprocess_summary(election_39_df, "39")
election_40_processed_df = preprocess_summary(election_40_df, "40")
election_41_processed_df = preprocess_summary(election_41_df, "41")
election_42_processed_df = preprocess_summary(election_42_df, "42")
election_43_processed_df = preprocess_summary(election_43_df, "43")
election_44_processed_df = preprocess_summary(election_44_df, "44")


summary_df = pd.concat([election_38_processed_df, election_39_processed_df, election_40_processed_df, 
           election_41_processed_df, election_42_processed_df, election_43_processed_df, election_44_processed_df], axis= 0)

summary_df.to_csv('data/power_bi/elections_summary.csv')
summary_df

,monetary_sum,num_donations,monetary_mean,monetary_min,monetary_max,electoral_district,election_number,election_year,contribution_cap,district_share
0,24370.740000,35,696.306857,250.000000,4319.400000,10003.0,38,2004,5000,0.282819
1,58275.000000,80,728.437500,250.000000,5000.000000,10003.0,38,2004,5000,0.676274
2,42247.500000,62,681.411290,230.000000,1000.000000,10004.0,38,2004,5000,0.830867
3,23620.000000,21,1124.761905,250.000000,5000.000000,10005.0,38,2004,5000,0.490194
4,12500.000000,22,568.181818,250.000000,1800.000000,10005.0,38,2004,5000,0.259417
...,...,...,...,...,...,...,...,...,...,...
503,19238.395415,29,663.392945,135.243553,2468.194842,35113.0,44,2021,1500,0.964407
504,2028.653295,2,1014.326648,676.217765,1352.435530,13001.0,44,2021,1500,0.951746
505,4503.610315,8,562.951289,108.194842,1420.057307,35073.0,44,2021,1500,1.000000
506,6472.756447,7,924.679492,430.074499,2366.762178,12002.0,44,2021,1500,0.239432
